<h2><a href="https://leetcode.com/problems/find-the-quiet-students-in-all-exams">1546. Find the Quiet Students in All Exams</a></h2><h3>Hard</h3><hr><p>Table: <code>Student</code></p>

<pre>
+---------------------+---------+
| Column Name         | Type    |
+---------------------+---------+
| student_id          | int     |
| student_name        | varchar |
+---------------------+---------+
student_id is the primary key (column with unique values) for this table.
student_name is the name of the student.</pre>

<p>&nbsp;</p>

<p>Table: <code>Exam</code></p>

<pre>
+---------------+---------+
| Column Name   | Type    |
+---------------+---------+
| exam_id       | int     |
| student_id    | int     |
| score         | int     |
+---------------+---------+
(exam_id, student_id) is the primary key (combination of columns with unique values) for this table.
Each row of this table indicates that the student with student_id had a score points in the exam with id exam_id.
</pre>

<p>&nbsp;</p>

<p>A <strong>quiet student</strong> is the one who took at least one exam and did not score the highest or the lowest score.</p>

<p>Write a solution&nbsp;to report the students <code>(student_id, student_name)</code> being quiet in all exams. Do not return the student who has never taken any exam.</p>

<p>Return the result table <strong>ordered</strong> by <code>student_id</code>.</p>

<p>The result format is in the following example.</p>

<p>&nbsp;</p>
<p><strong class="example">Example 1:</strong></p>

<pre>
<strong>Input:</strong> 
Student table:
+-------------+---------------+
| student_id  | student_name  |
+-------------+---------------+
| 1           | Daniel        |
| 2           | Jade          |
| 3           | Stella        |
| 4           | Jonathan      |
| 5           | Will          |
+-------------+---------------+
Exam table:
+------------+--------------+-----------+
| exam_id    | student_id   | score     |
+------------+--------------+-----------+
| 10         |     1        |    70     |
| 10         |     2        |    80     |
| 10         |     3        |    90     |
| 20         |     1        |    80     |
| 30         |     1        |    70     |
| 30         |     3        |    80     |
| 30         |     4        |    90     |
| 40         |     1        |    60     |
| 40         |     2        |    70     |
| 40         |     4        |    80     |
+------------+--------------+-----------+
<strong>Output:</strong> 
+-------------+---------------+
| student_id  | student_name  |
+-------------+---------------+
| 2           | Jade          |
+-------------+---------------+
<strong>Explanation:</strong> 
For exam 1: Student 1 and 3 hold the lowest and high scores respectively.
For exam 2: Student 1 hold both highest and lowest score.
For exam 3 and 4: Studnet 1 and 4 hold the lowest and high scores respectively.
Student 2 and 5 have never got the highest or lowest in any of the exams.
Since student 5 is not taking any exam, he is excluded from the result.
So, we only return the information of Student 2.
</pre>

In [2]:
from pyspark.sql import SparkSession
from pyspark.sql.types import StructType, StructField, IntegerType, StringType
from pyspark.sql.functions import col, min, max

# Initialize Spark Session
spark = SparkSession.builder.master("local").appName("QuietStudents").getOrCreate()

# Define Schema for Student Table
student_schema = StructType([
    StructField("student_id", IntegerType(), False),
    StructField("student_name", StringType(), False)
])

# Define Schema for Exam Table
exam_schema = StructType([
    StructField("exam_id", IntegerType(), False),
    StructField("student_id", IntegerType(), False),
    StructField("score", IntegerType(), False)
])

# Hardcoded Student Data
student_data = [
    (1, "Daniel"),
    (2, "Jade"),
    (3, "Stella"),
    (4, "Jonathan"),
    (5, "Will")
]

# Hardcoded Exam Data
exam_data = [
    (10, 1, 70),
    (10, 2, 80),
    (10, 3, 90),
    (20, 1, 80),
    (30, 1, 70),
    (30, 3, 80),
    (30, 4, 90),
    (40, 1, 60),
    (40, 2, 70),
    (40, 4, 80)
]

# Create DataFrames
students_df = spark.createDataFrame(student_data, schema=student_schema)
exams_df = spark.createDataFrame(exam_data, schema=exam_schema)

# Display DataFrames
print("Students Table:")
students_df.show()

print("Exams Table:")
exams_df.show()

Students Table:
+----------+------------+
|student_id|student_name|
+----------+------------+
|         1|      Daniel|
|         2|        Jade|
|         3|      Stella|
|         4|    Jonathan|
|         5|        Will|
+----------+------------+

Exams Table:
+-------+----------+-----+
|exam_id|student_id|score|
+-------+----------+-----+
|     10|         1|   70|
|     10|         2|   80|
|     10|         3|   90|
|     20|         1|   80|
|     30|         1|   70|
|     30|         3|   80|
|     30|         4|   90|
|     40|         1|   60|
|     40|         2|   70|
|     40|         4|   80|
+-------+----------+-----+



In [3]:
# Create temporary views for DataFrames
students_df.createOrReplaceTempView("Student")
exams_df.createOrReplaceTempView("Exam")

In [9]:
spark.sql("""
    SELECT 
          student_id, student_name
        FROM  (SELECT e.student_id,
                s.student_name,
                e.exam_id,
                CASE 
                    WHEN score = MAX(score) OVER (PARTITION BY exam_id) THEN 1
                    ELSE 0
                END as is_student_topper,
                CASE 
                    WHEN score = MIN(score) OVER (PARTITION BY exam_id) THEN 1
                    ELSE 0
                END as is_student_lowest
                FROM Exam e
                RIGHT JOIN Student s ON e.student_id = s.student_id
                WHERE e.exam_id IS NOT NULL
            ) temp
          GROUP BY student_id, student_name
          HAVING (CASE
            WHEN SUM(is_student_topper) = 0 AND SUM(is_student_lowest) = 0 THEN 1
            ELSE 0
          END) = 1
""").show()

+----------+------------+
|student_id|student_name|
+----------+------------+
|         2|        Jade|
+----------+------------+

